# Comparison of Kernels applied to a Banana Model


In [ ]:
import os
import numpy as np

from abcsmc.abcsmc import abcsmc
from abcsmc.models import BananaModel
from abcsmc.priors import UniformPrior
import abcsmc.pl as pl


In [ ]:
# ABC-SMC parameters
NTRIALS = 10
KERNEL_METHODS = ['locm', 'mvn', 'gaussian', 'uniform']
NPARTICLES = 800
EPS_SCHED = [160, 120, 80, 60, 40, 30, 20, 15, 10, 8, 6, 4, 3, 2, 1]

# Image output
imgdir = "../../out/images/nb_comparisons_banana"
os.makedirs(imgdir, exist_ok=True)

### Generate observed data $X_0$


In [ ]:
# Observed data
data = np.array([[0, 0]])

ndraws = len(data)

# Parameter indices and names used for plotting
pidx1 = 0
pidx2 = 1
pname1 = "$\\theta_1$"
pname2 = "$\\theta_2$"

# Priors
prior_theta1 = UniformPrior(-20, 20)
prior_theta2 = UniformPrior(-20, 20)
prior_list = [prior_theta1, prior_theta2]
plot_range = [[-20, 20], [-20, 20]]

# Model object
cov_const = [[1, 0], [0, 0.5]]
model = BananaModel(0, 0, cov=cov_const, ndraws=ndraws)

# Simulation function
def f_sim(particle):
    m = BananaModel(particle[0], particle[1], cov=cov_const, ndraws=ndraws)
    return m.generate_data(ndraws)

# Distance function
def f_dist(x):
    d = np.linalg.norm(x - data)
    assert np.ndim(d) == 0, "Bad distance function"
    return d


### Plot Analytic Posterior

In [ ]:
logposterior = False
pl.plot_posterior(
    model, data, prior_list,
    gridn=400,
    xlims=plot_range[0], ylims=plot_range[1],
    pname1=pname1, pname2=pname2,
    logposterior=logposterior,
    saveas=f"{imgdir}/analytic_posterior_plot.png",
    markersize=3,
)

print("Observed data D\n", data)

## Run ABC-SMC

In [ ]:
acc_results = {km:np.zeros([NTRIALS, len(EPS_SCHED)]) for km in KERNEL_METHODS}
for km in KERNEL_METHODS:
    for trialidx in range(NTRIALS):
        particles, weights, results_dict = abcsmc(
            nparticles=NPARTICLES, 
            nparams=2, 
            prior_list=prior_list, 
            niters=len(EPS_SCHED),
            sim_func=f_sim,
            dist_func=f_dist, 
            eps_sched=EPS_SCHED,
            min_eps=0, 
            kernel_method=km,
            track_all_perturbations = False,
            verbosity=0,
            disable_pbars=True,
        )
        acc_results[km][trialidx,:] = results_dict['acceptance_rates']


In [ ]:
pl.plot_kernel_comparison_results(
    acc_results, KERNEL_METHODS, EPS_SCHED,
    linewidth=3,
    title="Kernel Comparison (Ellipsoid Model)",
    saveas=f"{imgdir}/comparison_acc_rates.png",
)